In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
raw_csv_data = np.loadtxt('data_preprocessed.csv', delimiter=',')
inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [3]:
# shuffle
shuffled_indices = np.arange(inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = inputs_all[shuffled_indices]
shuffled_targets = targets_all[shuffled_indices]

In [4]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets)/test_samples_count)

1.0267558528428093
0.8918918918918919
0.9210526315789473


In [5]:
np.savez('data_train',inputs=train_inputs, targets=train_targets)
np.savez('data_validation',inputs=validation_inputs, targets=validation_targets)
np.savez('data_test',inputs=test_inputs, targets=test_targets)

# Loading the train, validation and test data as tensors

In [6]:
npz = np.load('data_train.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)


npz = np.load('data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)


# Creating the Neural Network

In [7]:
input_size = 12 # 12 predictors in data
output_size = 3 # 3 output labels
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #hidden layer 1
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #hidden layer 2
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #hidden layer 3
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer; we use softmax activation as we need probabilities as the output
    ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

batch_size = 50
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)

model.fit(train_inputs, train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2
          )



Epoch 1/100


6/6 - 4s - loss: 61.1375 - accuracy: 0.4047 - val_loss: 28.6259 - val_accuracy: 0.7297 - 4s/epoch - 716ms/step
Epoch 2/100
6/6 - 0s - loss: 39.0704 - accuracy: 0.4013 - val_loss: 24.3073 - val_accuracy: 0.1892 - 69ms/epoch - 12ms/step
Epoch 3/100
6/6 - 0s - loss: 29.7060 - accuracy: 0.4983 - val_loss: 30.9484 - val_accuracy: 0.0811 - 103ms/epoch - 17ms/step
Epoch 4/100
6/6 - 0s - loss: 20.6626 - accuracy: 0.2943 - val_loss: 15.2833 - val_accuracy: 0.7297 - 166ms/epoch - 28ms/step
Epoch 5/100
6/6 - 0s - loss: 19.9273 - accuracy: 0.4950 - val_loss: 18.3695 - val_accuracy: 0.1892 - 65ms/epoch - 11ms/step
Epoch 6/100
6/6 - 0s - loss: 10.2700 - accuracy: 0.4314 - val_loss: 3.9321 - val_accuracy: 0.7297 - 98ms/epoch - 16ms/step
Epoch 7/100
6/6 - 0s - loss: 14.1492 - accuracy: 0.4247 - val_loss: 12.8375 - val_accuracy: 0.1351 - 145ms/epoch - 24ms/step
Epoch 8/100
6/6 - 0s - loss: 15.2710 - accuracy: 0.4749 - val_loss: 15.3605 - val_accuracy: 0.2432 - 76ms/epoch - 13ms/step


In [8]:
# testing

test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('test loss: {0:.2f} test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

2/2 [==============================] - 0s 17ms/step - loss: 12.0675 - accuracy: 0.3421
test loss: 12.07 test accuracy: 34.21%
